# Compound Pool Economics - The Graph

In [47]:
# cadCAD standard dependencies

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

# cadCAD global simulation configuration list
from cadCAD import configs

In [48]:
# Additional dependencies

# For parsing the data from the API
import json
# For downloading data from API
import requests as req
# For generating random numbers
import math
# For analytics
import pandas as pd
# For visualization
import plotly.express as px

# Setup / Preparatory Steps

## Query the Balancer subgraph for a UNI-BAL pool

In [49]:
# You can explore the subgraph at https://thegraph.com/hosted-service/subgraph/graphprotocol/compound-v2
API_URI = 'https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2'

# Query for retrieving the history of swaps on a BAL <> UNI 50-50 pool
GRAPH_QUERY = '''
{
  markets{
        borrowRate
        supplyRate
        totalBorrows
        totalSupply
        exchangeRate
  }
}
'''
'''
    borrowRate
    cash
    collateralFactor
    exchangeRate
    interestRateModelAddress
    name
    reserves
    supplyRate
    symbol
    id
    totalBorrows
    totalSupply
    underlyingAddress
    underlyingName
    underlyingPrice
    underlyingSymbol
    reserveFactor
    underlyingPriceUSD
'''

# Retrieve data from query
JSON = {'query': GRAPH_QUERY}
r = req.post(API_URI, json=JSON)
graph_data = json.loads(r.content)['data']

print("Print first 200 characters of the response")
print(r.text[:200])

Print first 200 characters of the response
{"data":{"markets":[{"borrowRate":"0.0095037908380992","supplyRate":"0.0013548306031776","totalBorrows":"4964.751249095357457239","totalSupply":"1288952.38506475","exchangeRate":"0.020264400207980977"


## Data Wrangle the Data

In [50]:
raw_df = pd.DataFrame(graph_data['markets'])

raw_df.head(5)

,borrowRate,supplyRate,totalBorrows,totalSupply,exchangeRate
0,0.0095037908380992,0.0013548306031776,4964.751249095357457239,1288952.38506475,0.020264400207980977
1,0.0113316268084416,0.0023755066712064,523732.489980387674823315,111793064.40064639,0.02067151926661021
2,0.0353730678998976,0,56.182564705386870913,54704.86438779,0.020040659236629971
3,0.0487296703804032,0.004666624557984,543213.046131158710280341,209351183.79462266,0.0203210734450647
4,0.0261586483598304,0.01265908535568,315810062.43341,26557260782.87954492,0.0227298929171069


In [51]:
# Clean the data:
# 1. convert the raw timestamps to Python DateTime objects
# 2. make the token flow values numerical
# 3. order by time
df = (raw_df.assign(totalBorrows=lambda df: pd.to_numeric(df.totalBorrows))
            .assign(totalSupply=lambda df: pd.to_numeric(df.totalSupply))
            .assign(borrowRate=lambda df: pd.to_numeric(df.borrowRate))
            .assign(supplyRate=lambda df: pd.to_numeric(df.supplyRate))
            .assign(exchangeRate=lambda df: pd.to_numeric(df.exchangeRate))
            .reset_index()
      )

df.head(5)

,index,borrowRate,supplyRate,totalBorrows,totalSupply,exchangeRate
0,0,0.009504,0.001355,4.964751e+03,1.288952e+06,0.020264
1,1,0.011332,0.002376,5.237325e+05,1.117931e+08,0.020672
2,2,0.035373,0.000000,5.618256e+01,5.470486e+04,0.020041
3,3,0.048730,0.004667,5.432130e+05,2.093512e+08,0.020321
4,4,0.026159,0.012659,3.158101e+08,2.655726e+10,0.022730


# Modelling

## 1. State Variables

In [52]:
initial_state = {
    'lender_APY': 0.0,
    'borrower_rate': 0.0,
    'utilization_rate': 0.0,
    'exchange_rate': 0.0
}
initial_state

{'lender_APY': 0.0,
 'borrower_rate': 0.0,
 'utilization_rate': 0.0,
 'exchange_rate': 0.0}

## 2. System Parameters

In [53]:
# Transform the swap history data frame into a {timestep: data} dictionary
# Turning the df into a dictionary form
df_dict = df.to_dict(orient='index')

system_params = {
    'new_df': [df_dict],
    
    # Transaction fees being applied to the input token
    'exchange_rate': ['exchangeRate']
}

# Element for timestep = 3

## 3. Policy Functions

In [54]:
def p_rates(params, substep, state_history, previous_state):
    """
    Calculate cumulative transaction fees & swaps
    from a swap event
    """
    t = previous_state['timestep']
    
    # Data for this timestep
    ts_data = params['new_df'][t]    
    

    lender_APY = ts_data['supplyRate']
 
    borrower_rate = ts_data['borrowRate']
    
    exchange_rate = params['exchange_rate']
    
    total_borrowed = ts_data['totalBorrows']
    TVL = ts_data['totalSupply']

    #utilization_rate = total_borrowed / TVL * 100
    try:
        utilization_rate = pd.to_numeric(total_borrowed) / pd.to_numeric(TVL) * 100
    except ZeroDivisionError:
        utilization_rate = 0
        
    #exchange_rate1 = swap_in * params['exchange_rate']

    return {'lender_APY': lender_APY,
            'borrower_rate': borrower_rate,
            'exchange_rate': exchange_rate,
            'utilization_rate': utilization_rate}

In [55]:
print(df.dtypes)

index             int64
borrowRate      float64
supplyRate      float64
totalBorrows    float64
totalSupply     float64
exchangeRate    float64
dtype: object


## 4. State Update Functions

In [56]:
def s_lender_APY(params,
                      substep,
                      state_history,
                      previous_state,
                      policy_input):
    value = policy_input['lender_APY']
    return ('lender_APY', value)

def s_borrower_APY(params,
                              substep,
                              state_history,
                              previous_state,
                              policy_input):
    value = policy_input['borrower_rate']
    #fee = policy_input['fee_UNI']
    #value = previous_state['cumulative_fee_UNI'] + fee 
    return ('borrower_rate', value)


def s_utilization_rate(params,
                              substep,
                              state_history,
                              previous_state,
                              policy_input):
    value = policy_input['utilization_rate']
    #fee = policy_input['fee_BAL']
    #value = previous_state['cumulative_fee_BAL'] + fee 
    return ('utilization_rate', value)

def s_exchange_rate(params,
                              substep,
                              state_history,
                              previous_state,
                              policy_input):
    value = policy_input['exchange_rate']
    #fee = policy_input['fee_BAL']
    #value = previous_state['cumulative_fee_BAL'] + fee 
    return ('exchange_rate', value)

## 5. Partial State Update Blocks

In [57]:
partial_state_update_blocks = [
    {
        'policies': {
            'policy_rates': p_rates
        },
        'variables': {
            's_lender_APY': s_lender_APY,
            's_borrower_rate': s_borrower_APY,
            's_exchange_rate': s_exchange_rate,
            's_utilization_rate': s_utilization_rate
        }
    }
]

# Simulation

## 6. Configuration

In [58]:
sim_config = config_sim({
    "N": 1, # the number of times we'll run the simulation ("Monte Carlo runs")
    "T": range(len(df)), # the number of timesteps the simulation will run for
    "M": system_params # the parameters of the system
})

In [59]:
del configs[:] # Clear any prior configs

In [60]:
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

## 7. Execution

In [61]:
exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (19, 2, 1, 4)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
Total execution time: 0.01s


## 8. Output Preparation

In [62]:
simulation_result = pd.DataFrame(raw_result)
simulation_result.head(5)

,lender_APY,borrower_rate,utilization_rate,exchange_rate,simulation,subset,run,substep,timestep
0,0.000000,0.000000,0.000000,0.0,0,0,1,0,0
1,0.001355,0.009504,0.385177,exchangeRate,0,0,1,1,1
2,0.002376,0.011332,0.468484,exchangeRate,0,0,1,1,2
3,0.000000,0.035373,0.102701,exchangeRate,0,0,1,1,3
4,0.004667,0.048730,0.259475,exchangeRate,0,0,1,1,4


## 9. Analysis

In [65]:
# Visualize how much transaction fees were paid over time on each token
print("High supply of lenders → Low utilization rate → Lower lender APY")
print("High demand for borrowing → High utilization rate → Higher borrower rates")
graph = px.line(simulation_result,
           x='timestep',
           y=['lender_APY', 'borrower_rate'],
              #, 'exchange_rate'],
              #, 'utilization_rate'],
           title='Compound Pool Economics',
           facet_row='subset')

graph.update_layout(yaxis=dict(tickformat="%", hoverformat="%.2f%"))
graph1 = graph.update_yaxes(hoverformat=".2%")
graph1


High supply of lenders → Low utilization rate → Lower lender APY
High demand for borrowing → High utilization rate → Higher borrower rates


In [66]:
ur_graph = px.line(simulation_result,
           x='timestep',
           y=['utilization_rate'],
           facet_row='subset')

ur_graph = ur_graph.update_layout(yaxis=dict(tickformat="%", hoverformat="%.2f%"))
ur_graph1 = ur_graph.update_yaxes(hoverformat=".2%")
ur_graph1